# _Download_ das Demonstrações Financeiras das empresas listadas na Bolsa B3

__Base de dados:__

Dados CVM: https://dados.cvm.gov.br/

Formulário de Informações Trimestrais (ITR): https://dados.cvm.gov.br/dataset/cia_aberta-doc-itr

Histórico desde 2011: 

- ITR: https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/
- DFP: https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/

__Bibliotecas importadas:__

In [2]:
import os
import requests
import wget
import pandas as pd
from zipfile import ZipFile
from bs4 import BeautifulSoup

In [3]:
ROOT_DIR = os.getcwd().replace('src', '')
zips_dir = os.path.join(ROOT_DIR, 'data', 'dfcs', 'tmp', 'zips')
csvs_dir = os.path.join(ROOT_DIR, 'data', 'dfcs', 'tmp', 'csvs')

In [4]:
for tipo in ['itr', 'dfp']:
    database_url = f'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/{tipo.upper()}/DADOS/'
    page = requests.get(database_url)
    soup = BeautifulSoup(page.text, 'html.parser')
    pre = soup.find('pre')
    arquivos_zip = [a['href'] for a in pre.find_all('a')[1:]]
    zips_dir = os.path.join(ROOT_DIR, 'data', 'dfcs', 'tmp', 'zips')
    for arquivo in arquivos_zip:
        wget.download(database_url + arquivo, zips_dir)
        ZipFile(os.path.join(zips_dir, arquivo), 'r').extractall(csvs_dir)

100% [........................................................] 244725 / 244725

In [7]:
for tipo in ['itr', 'dfp']:
    nomes = ['BPA_con', 'BPP_con', 'DRE_con']
    for nome in nomes:
        arquivo = pd.DataFrame()
        ini, fim = 2017, 2021
        for ano in range(ini, fim + 1):
            df = pd.read_csv(os.path.join(csvs_dir, f'{tipo}_cia_aberta_{nome}_{ano}.csv'), sep=';', decimal=',', encoding='ISO-8859-1')
            arquivo = pd.concat([arquivo, df])
        arquivo.to_csv(os.path.join(ROOT_DIR, 'data', 'dfcs', f'{tipo}_cia_aberta_{nome}_{ini}-{fim}.csv'), index=False)